# Demonstration of McStasScript
This file demonstrates how McStasScript can be used to run McStas from a python environment in a userfreindly manner.

In [ ]:
from mcstasscript.interface import instr, plotter, functions

# Creating the instance of the class, insert path to mcrun and to mcstas root directory
Instr = instr.McStas_instr("jupyter_demo")

In [ ]:
Instr.show_components() # Shows available McStas component categories in current installation

In [ ]:
Instr.show_components("sources") # Display all McStas source components 

In [ ]:
Instr.component_help("Source_simple") # Displays help on the Source_simple component

In [ ]:
source = Instr.add_component("Source", "Source_simple") # Adds an instance of Source_simple

In [ ]:
# Lets add a parameter to the instrument to control the wavelength of the source
wavelength = Instr.add_parameter("double", "wavelength", value=3,
                                 comment="[AA] Wavelength emmited from source")
source.xwidth = 0.06
source.yheight = 0.08;
source.dist = 2
source.focus_xw = 0.05
source.focus_yh = 0.05
source.lambda0 = wavelength # Can provide a parameter object, in this way spelling is checked
source.dlambda = 0.05
source.flux = 1E8

In [ ]:
print(source) # Verify that the information is correct

In [ ]:
guide = Instr.add_component("Guide", "Guide_gravity", AT=[0,0,2], RELATIVE="Source")
guide.set_comment="Beam extraction and first guide piece"

In [ ]:
guide.show_parameters() # Lets view the parameters available in our guide component

In [ ]:
guide.set_parameters(w1=0.05, w2=0.05, h1=0.05, h2=0.05, l=8, m=3.5, G=-9.2)

In [ ]:
print(guide) # Verify the information on this component is correct

In [ ]:
# Add a sample to the instrument
sample = Instr.add_component("sample", "PowderN", AT=[0, 0, 9], RELATIVE="Guide") 

In [ ]:
# Set parameters corresponding to a copper cylinder
sample.radius = 0.015
sample.yheight = 0.05
sample.reflections = '"Cu.laz"'

In [ ]:
Instr.show_components("monitors") # Monitors are needed to record information

In [ ]:
# Add 4PI detector to detect all neutrons
sphere = Instr.add_component("PSD_4PI", "PSD_monitor_4PI", RELATIVE="sample")

In [ ]:
sphere.nx = 300
sphere.ny = 300
sphere.radius = 1
sphere.restore_neutron = 1
sphere.filename = '"PSD_4PI.dat"' # filenames need printed quotes, use both ' and "
print(sphere) # Verify that monitors have filenames that are strings when printed

In [ ]:
# Add PSD monitor to see the direct beam after the sample
PSD = Instr.add_component("PSD", "PSD_monitor", AT=[0,0,1], RELATIVE="sample") 
PSD.xwidth = 0.1
PSD.yheight = 0.1
PSD.nx = 200
PSD.ny = 200
PSD.filename = '"PSD.dat"'
PSD.restore_neutron = 1

In [ ]:
L_mon = Instr.add_component("L_mon", "L_monitor", RELATIVE="PSD")

In [ ]:
# Since the wavelength is an instrument parameter, it can be used when setting parameters
L_mon.Lmin = "wavelength - 0.1"
L_mon.Lmax = "wavelength + 0.1"; L_mon.nL = 150
L_mon.xwidth = 0.1
L_mon.yheight = 0.1
L_mon.filename = '"wave.dat"'
L_mon.restore_neutron = 1
L_mon.comment = "Wavelength monitor for narrow range"

In [ ]:
print(L_mon)

In [ ]:
Instr.print_components() # Lets get an overview of the instrument so far

In [ ]:
Instr.show_parameters()

# Running the McStas instrument
Now we have assembled an instrument and it is time to perform a simulation

In [ ]:
# output_path specifies the foldername, if it already exists an index is added
Instr.settings(output_path="jupyter_demo", mpi=4, ncount=2E7)
Instr.show_settings() # Check settings are correct

In [ ]:
# Input parameters are set with set_parameters
Instr.set_parameters(wavelength=1.5)
Instr.show_parameters()

In [ ]:
# The simulation is performed by calling backengine()
Instr.backengine()

# The data is retrieved from the data attribute
data = Instr.data

## Working with the returned data
The returned data object is a list of McStasData objects, each containing the results from a monitor.
These data objects also contain preferences for how they should be plotted if this is done automatically.

In [ ]:
wavelength_data = functions.name_search("L_mon", data)
wavelength_intensity = wavelength_data.Intensity
wavelength_xaxis = wavelength_data.xaxis

for index in range(70,75):
    print([wavelength_xaxis[index], wavelength_intensity[index]])

## Plotting the returned data
The plot options looks at some metadata in the McStasData for plotting preferences. For this reason these options can be adjusted for individual data files instead of complex syntax for the plotting command.

In [ ]:
# Adjusting PSD_4PI plot
functions.name_plot_options("PSD_4PI", data, log=1, colormap="hot", orders_of_mag=5)

plot = plotter.make_sub_plot(data) # Making subplot of our monitors